# Getting Started with ESA++

ESA++ wraps PowerWorld's SimAuto COM interface into Pythonic indexing.
This notebook walks through opening a case, reading and writing data,
and running a power flow.

## Opening a Case

Create a `GridWorkBench` and import the component classes you need.

```python
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Load, Branch

wb = GridWorkBench("path/to/case.pwb")
```

In [2]:
# This cell is hidden in the documentation.
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Load, Branch
import numpy as np
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

'open' took: 6.1796 sec


## Reading Data

Use `wb[ComponentType, fields]` to query data. A single field returns a
two-column DataFrame (keys + values); a list of fields returns all
requested columns with keys prepended automatically.

In [3]:
wb[Bus, "BusPUVolt"].head()

,BusNum,BusPUVolt
0,1,0.993545
1,2,0.991225
2,3,0.984548
3,4,0.978800
4,5,0.988985


In [4]:
wb[Gen, ["GenMW", "GenMVR", "GenStatus"]].head()

,BusNum,GenID,GenMVR,GenMW,GenStatus
0,2,1,0.80000,2.500000,Closed
1,2,2,0.80000,2.500000,Closed
2,2,3,0.80000,2.500000,Closed
3,2,4,0.80000,2.500000,Closed
4,23,1,0.04408,69.274741,Closed


The result is a regular pandas DataFrame, so standard filtering works:

In [5]:
gens = wb[Gen, ["GenMW", "GenMVR", "GenStatus"]]
online_gens = gens[gens["GenStatus"] == "Closed"]
online_gens.head()

,BusNum,GenID,GenMVR,GenMW,GenStatus
0,2,1,0.80000,2.500000,Closed
1,2,2,0.80000,2.500000,Closed
2,2,3,0.80000,2.500000,Closed
3,2,4,0.80000,2.500000,Closed
4,23,1,0.04408,69.274741,Closed


In [6]:
loads = wb[Load, ["LoadMW", "LoadMVR"]]
print(f"Total system load: {loads['LoadMW'].sum():.1f} MW")

Total system load: 1136.3 MW


## Writing Data

Assign with the same syntax. A scalar broadcasts to every row.

In [7]:
wb[Gen, "GenMW"] = 100.0
wb[Gen, "GenMW"].head()

,BusNum,GenID,GenMW
0,2,1,100.0
1,2,2,100.0
2,2,3,100.0
3,2,4,100.0
4,23,1,100.0


## Power Flow

`pflow()` solves the AC power flow and returns complex bus voltages.

In [8]:
V = wb.pflow()
V.head()

0    0.981083+0.147067j
1    1.000984+0.082193j
2    0.989441-0.014760j
3    0.985577+0.023944j
4    0.980118+0.118430j
dtype: complex128

In [9]:
vmag = np.abs(V)
print(f"Voltage range: {vmag.min():.4f} – {vmag.max():.4f} pu")
print(f"Buses below 0.98 pu: {(vmag < 0.98).sum()}")

Voltage range: 0.9818 – 1.0914 pu
Buses below 0.98 pu: 0


## Branch Data

Query branch loading the same way. The result is a plain DataFrame
you can sort, filter, or export as needed.

In [10]:
branches = wb[Branch, ["BusNum", "BusNum:1", "LineMVA", "LinePercent"]]
branches.sort_values("LinePercent", ascending=False).head()

,BusName_NomVolt:1,BusNum,BusNum:1,LineCircuit,LineMVA,LinePercent
72,COGEN69_69.00,24,36,1,378.250980,776.696077
67,SCHOFIELD69_69.00,23,34,1,318.164998,427.641116
68,SCHOFIELD69_69.00,23,34,2,318.164998,427.641116
18,EWA BEACH69_69.00,2,26,1,187.153468,303.328158
19,EWA BEACH69_69.00,2,26,2,187.153468,303.328158
